<a id='top'></a>
# Table of contents
- [Random Generator](#randomgenerator)
- [Environment](#environment)
- [Infrastructure Object](#infrastructureobject)
- [Processing Station](#processingstation)
- [Machine](#machine)
- [Buffer](#buffer)
- [Source](#source)
- [Dispatcher](#dispatcher)
- [Operation](#operation)
- [Job](#job)
- [Logic Test](#logic_test)

# To-Do:
- [ ] capacity deadlocks between buffers and machines
    - [x] ~~remove deadlocks by counting the associated machines and comparing the counter to the buffer's capacity~~ (**no capacities > 1 for processing stations**)
    - **problem persists: if predecessor systems produce faster than the target processing stations, they fill the buffer until it is filled --> result == deadlock again**
    - shared buffers: if only used for machine groups --> no problem because there are no circles between these machines
    - [ ] look into problems where processing stations have a capacity greater than 1
- [ ] description of process logic for generation and entry of jobs
- [ ] add setting of job/machine states in machine logic
    - using state update function which combine all necessary state update calls
- [ ] add status info to job DB? (waiting, processing, ...) (disposable = waiting?)
    - advantage:
        - one central DB with all information, no cluttered information
        - simple filtering for jobs by current status incl. disposable jobs
- [ ] add sources and sinks
    - source: generates new job with given intervals
    - sink: destroys jobs and finalises data collection
- [ ] add physical buffers
- [ ] add logistic objective values:
    - WIP
    - lead time
- [ ] add Gantt chart visualisation for debugging
    - [x] ~~after simulation run~~
    - [ ] during simulation run
- [ ] logic/ interface for generation of multiple jobs
    - using dispatcher or source?
    - interface: design + properties
- [ ] add priority rules
- [ ] add machine groups (parallel machines)
- [ ] I/O functions for elements
- [ ] initialisation of the model with pre-defined state information
- [ ] self-marking as disposable by jobs
    - ==*check if still necessary*==
    - includes demarking
    - currently only addition implemented
- [ ] add machine groups
    - logic behind communicating machine groups
    - registration of machines in groups
- [x] ~~register job object in dispatcher~~
- [x] ~~register operations in dispatcher~~
- [x] ~~add uniqueness check for custom IDs in resource objects (only interface to user)~~
- [x] ~~operations list of jobs as deque~~
- [x] ~~add generic infrastructure class from which infrastructure objects are derived~~
- [x] ~~implement routing logic in objects~~
- [x] ~~add operation starting and end points~~

- [ ] tracking disjunctive graph model

### Logic behind simulation model approach
*Model of Resource and Load*:
- system consists of physical objects, also called infrastructure
    - each element can be considered as encapsulated resource
- stress can be put on each system by occupying resources, also called load
    - definition of load depends on system type and modelling, e.g. production jobs for production systems or customers for cashiers in a shop
- load objects are called ***load unit***

*Guiding Priciples*:
- **load objects** can only be spatially and temporally modified by **resources**
    - only resource objects can put load objects on other resources and change their state
    - load objects **contain the necessary information** which is essential for their further processing

################

Logic of ``Lang et al.: Modeling Production Scheduling Problems as Reinforcement Learning Environments based on Discrete-Event Simulation and OpenAI Gym``
- whole routing logic is implemented in a collaborative manner between resources and load objects
    - each load object puts itself in a associated queue
    - therefore load objects can change their *states* and *location* by theirown
- **violates resource-load model: no load object can change its state without a associated resource**

In [520]:
import numpy as np
import numpy.typing as npt
from numpy.random._generator import Generator
import random
import simpy
import salabim as sim
from salabim import Queue, State
from typing import TypeAlias, Iterable, Iterator, Any
from collections import OrderedDict, deque
from operator import attrgetter
from functools import lru_cache
import logging
import sys
import pandas as pd
from pandas import DataFrame, Series
import plotly.express as px
from plotly.graph_objs._figure import Figure
from __future__ import annotations

sim.yieldless(False)

# type aliases
NPRandomGenerator: TypeAlias = Generator
SimPyEnv: TypeAlias = simpy.core.Environment
SalabimEnv: TypeAlias = sim.Environment
EnvID: TypeAlias = int
#JobID: TypeAlias = int
#OpID: TypeAlias = int
ObjectID: TypeAlias = int
### [CHANGE] Replace MachineID as CustomID
MachineID: TypeAlias = int | str
CustomID: TypeAlias = int | str
#InfstructObj: TypeAlias = object # better naming in future
PlotlyFigure: TypeAlias = Figure

# forward reference, referenced before assignment
#Job: TypeAlias = 'Job'
#Dispatcher: TypeAlias = 'Dispatcher'

# logging
# IPython compatibility
logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
LOGGING_LEVEL = 'DEBUG'
LOGGING_LEVEL_ENV = 'ERROR'
LOGGING_LEVEL_DISPATCHER = 'DEBUG'
LOGGING_LEVEL_SOURCES = 'DEBUG'
LOGGING_LEVEL_PRODSTATIONS = 'DEBUG'
LOGGING_LEVEL_JOBS = 'ERROR'
LOGGING_LEVEL_OPERATIONS = 'ERROR'


logger = logging.getLogger('base')
logger.setLevel(LOGGING_LEVEL)
logger_env = logging.getLogger('env')
logger_env.setLevel(LOGGING_LEVEL_ENV)
logger_dispatcher = logging.getLogger('dispatcher')
logger_dispatcher.setLevel(LOGGING_LEVEL_DISPATCHER)
logger_sources = logging.getLogger('sources')
logger_sources.setLevel(LOGGING_LEVEL_SOURCES)
logger_prodStations = logging.getLogger('prodStations')
logger_prodStations.setLevel(LOGGING_LEVEL_PRODSTATIONS)

logger_jobs = logging.getLogger('jobs')
logger_jobs.setLevel(LOGGING_LEVEL_JOBS)
logger_operations = logging.getLogger('operations')
logger_operations.setLevel(LOGGING_LEVEL_OPERATIONS)



INF = float('inf')
FAIL_DELAY = 20 # time after a store request is failed

<a id='randomgenerator'></a>

In [2]:
class RandomJobGenerator(object):
    
    def __init__(
        self,
        seed: int = 42,
    ) -> None:
        """
        seed: seed value for random number generator
        """
        self._np_rnd_gen: NPRandomGenerator = np.random.default_rng(seed=seed)
        
    def gen_rnd_JSSP_inst(
        self,
        n_jobs: int,
        n_machines: int,
    ) -> tuple[npt.NDArray[np.uint16], npt.NDArray[np.uint16]]:
        """
        Generates random job shop instance with given number of and machines
        - each job on all machines
        - max processing time = 9
        
        Output:
        n_jobs: number of jobs
        n_machines: number of machines
        n_tasks: number of tasks
        mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
        """
        # generate random process time matrix shape=(n_jobs, n_machines)
        mat_ProcTimes = self._np_rnd_gen.integers(1, 10, size=(n_jobs,n_machines), dtype=np.uint16)
        
        # generate randomly shuffled job machine combinations
        # machine IDs from 1 to n_machines
        temp = np.arange(0, n_machines, step=1, dtype=np.uint16)
        temp = np.expand_dims(temp, axis=0)
        # repeat dummy line until number n_jobs is reached
        temp = np.repeat(temp, n_jobs, axis=0)
        # randomly permute the machine indices job-wise
        mat_JobMachID = self._np_rnd_gen.permuted(temp, axis=1)
        
        # generate operation ID matrix
        # not mandatory because operations are registered in the environment's dispatcher
        n_ops = n_jobs * n_machines
        temp2 = np.arange(0, (n_ops), step=1, dtype=np.uint16)
        mat_OpID = temp2.reshape(n_jobs, -1)
        
        return mat_ProcTimes, mat_JobMachID
    
    def gen_rnd_job(
        self,
        n_machines: int,
    ) -> tuple[npt.NDArray[np.uint16], npt.NDArray[np.uint16]]:
        """
        Generates random job shop instance with given number of jobs and machines'
        - each job on all machines
        - max processing time = 9
        
        Output:
        mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)

        """
        # generate random process time matrix shape=(n_machines)
        mat_ProcTimes = self._np_rnd_gen.integers(1, 10, size=n_machines, dtype=np.uint16)
        
        # generate randomly shuffled job machine combinations
        # machine IDs from 1 to n_machines
        temp = np.arange(0, n_machines, step=1, dtype=np.uint16)
        # randomly permute the machine indices job-wise
        mat_JobMachID = self._np_rnd_gen.permuted(temp)
        
        return mat_ProcTimes, mat_JobMachID
    

In [3]:
job_generator = RandomJobGenerator()

In [4]:
mat_ProcTimes, mat_JobMachID = job_generator.gen_rnd_JSSP_inst(2,3)

In [5]:
mat_ProcTimes

array([[2, 1, 9],
       [7, 9, 6]], dtype=uint16)

In [6]:
mat_JobMachID

array([[2, 0, 1],
       [0, 1, 2]], dtype=uint16)

##### Brainstorming of ways to import machine names and IDs
*Case 1: only names are given*
- read machine names and assert IDs to them
- build data structure with name and ID bundled (maybe as property of a machine class)

*Case 2: IDs are given*
- only building data structure with name and ID bundled

*Data Structure:*
- if only mapping of two pairs in each direction (lookup ID or lookup machine name)
    - bi-directional dictionary

**for resource objects:**
- CustomID may not be of type 'None' because the custom identifiers are the only interface to the end user (EnvID are solely handled inernally)
- add checking for uniqueness of custom identifiers necessary, else the mapping of different objects could be ambiguous
- jobs and operations can use ambiguous custom IDs --> custom IDs can still be None

- dedicated environment class with information on associated resources and jobs
- maybe add possibility of using subsystems (bundle of resources with unique identifiers)

<a id='environment'></a>
**Salabim Env**

In [365]:
class SimulationEnvironment(sim.Environment):
    
    def __init__(
        self,
        **kwargs,
    ) -> None:
        """
        
        """
        super().__init__(**kwargs)
        
        # resource data base as simple Pandas DataFrame
        self._infstruct_prop: dict[str, type] = {
            'env_id': int,
            'custom_id': object,
            'resource': object,
            'name': str,
            'res_type': str,
        }
        self._res_db: DataFrame = pd.DataFrame(columns=list(self._infstruct_prop.keys()))
        self._res_db: DataFrame = self._res_db.astype(self._infstruct_prop)
        self._res_lookup_props: set[str] = set(['env_id', 'custom_id', 'name'])
        
        # env identifiers
        self._id_counter: EnvID = 0
        self._res_custom_identifiers: set[str | int] = set()
        
        # job dispatcher
        self._dispatcher_registered: bool = False
        self._dispatcher: Dispatcher = None
        
        # counter for processing stations (machines, assembly, etc.)
        self.num_proc_stations: int = 0
        ############## LEGACY CODE
        ### legacy code, changed approach to tabular data structure
        """
        self.id_counter: EnvID = 0
        self.resources: dict[EnvID, object] = dict()
        self._custom_identifiers: set[CustomID] = set()
        self._custom_from_env_ids: dict[EnvID, CustomID] = dict()
        self._custom_to_env_ids: dict[CustomID, EnvID] = dict()
        """
        self._ultimative_test = pd.DataFrame(columns=['test1', 'test2'])
        
    def _obtain_env_id(self) -> EnvID:
        """Simple counter function for managing environment IDs"""
        # assign id and set counter up
        env_id = self._id_counter
        self._id_counter += 1
        
        return env_id
    
    def register_dispatcher(
        self,
        dispatcher: Dispatcher,
    ) -> EnvID:
        """
        Registers a dispatcher instance for the environment. Only one instance per environment is allowed.
        returns: EnvID for the dispatcher instance
        """
        # obtain env_id
        env_id = self._obtain_env_id()
        
        if not self._dispatcher_registered:
            self._dispatcher = dispatcher
            self._dispatcher_registered = True
            logger_env.info(f"Successfully registered dispatcher with EnvID {env_id}")
        else:
            raise AssertionError("There is already a registered dispatcher instance \
                                 Only one instance per environement is allowed.")
        
        return env_id
    
    @property
    def dispatcher(self) -> Dispatcher:
        """obtain the current registered dispatcher instance of the environment"""
        if self._dispatcher is None:
            raise ValueError("No Dipsatcher instance registered.")
        else:
            return self._dispatcher
    
    def register_resource(
        self,
        obj: InfrastructureObject,
        custom_identifier: CustomID,
        name: str | None,
    ) ->  tuple[EnvID, str]:
        """
        registers an infrastructure object in the environment by assigning an unique id and 
        adding the object to the associated resources of the environment
        
        obj: env resource = instance of a subclass of InfrastructureObject
        custom_identifier: user defined identifier
        name: custom name of the object, \
            default: None
        returns:
            env_id: assigned env ID
        """
        # check for uniqueness of custom_identifier
        # type security
        if not isinstance(custom_identifier, (str, int)):
            raise TypeError("Custom identifier must be of type STR or INT")
        # create check value
        if isinstance(custom_identifier, str):
            # remove capital letters for checking
            check_val = custom_identifier.lower()
        else:
            check_val = custom_identifier
        
        # check if value already exists
        if check_val in self._res_custom_identifiers:
            raise ValueError(f"The custom identifier {custom_identifier} provided already exists, \
                            but has to be unique.")
        else:
            self._res_custom_identifiers.add(check_val)
        
        # obtain env_id
        env_id = self._obtain_env_id()
        
        # count number of machines
        if isinstance(obj, ProcessingStation):
            self.num_proc_stations += 1
        
        # custom name
        if name is None:
            name = f'{type(obj).__name__}_env_{env_id}'
        
        # new entry for resource data base
        new_entry: DataFrame = pd.DataFrame({
                                'env_id': [env_id],
                                'custom_id': [custom_identifier],
                                'resource': [obj],
                                'name': [name],
                                'res_type': [obj.res_type]})
        new_entry: DataFrame = new_entry.astype(self._infstruct_prop)
        self._res_db = pd.concat([self._res_db, new_entry], ignore_index=True)
        
        logger_env.info(f"Successfully registered object with EnvID {env_id} and name {name}")
        
        return env_id, name
    
    @property
    def res_db(self) -> DataFrame:
        """obtain a current overview of registered objects in the environment"""
        return self._res_db

    #@lru_cache(maxsize=200)
    def get_res_obj_by_prop(
        self,
        property: str, 
        val: EnvID | CustomID | str,
        target_prop: str = 'resource',
    ) -> InfrastructureObject:
        """
        obtain a resource object from the environment by its property and corresponding value
        properties: env_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._res_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._res_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        temp1: Series = self._res_db.loc[self._res_db[property] == val, target_prop]
        # check for empty search result, at least one result necessary
        if len(temp1) == 0:
            raise IndexError(f"There were no resources found for the property '{property}' \
                            with the value '{val}'")
        # check for multiple entries with same prop-value pair
        ########### PERHAPS CHANGE NECESSARY
        ### multiple entries but only one returned --> prone to errors
        elif len(temp1) > 1:
            # warn user
            logger_env.warning(f"CAUTION: There are multiple resources which share the \
                        same value '{val}' for the property '{property}'. \
                        Only the first entry is returned.")
        
        return temp1.iat[0]
    
    
        

[Jump to top](#top)

---
### **Infrastructure Objects**

*Adapting machine class:*
- user defined process function
- env related stuff in generic class

<a id='infrastructureobject'></a>

In [576]:
class InfrastructureObject(sim.Component):
    
    def __init__(
        self,
        env: SimulationEnvironment,
        custom_identifier: CustomID,
        name: str | None = None,
        capacity: float = INF,
        **kwargs,
    ) -> None:
        """
        env: simulation environment in which the infrastructure object is embedded
        custom_identifier: unique user-defined custom ID of the given object \
            necessary for user interfaces
        capacity: capacity of the infrastructure object, if multiple processing \
            slots available at the same time > 1, default=1
        """
        # assert machine information and register object in the environment
        self._env_id, name = env.register_resource(
                                obj=self, custom_identifier=custom_identifier,
                                name=name)
        self.custom_identifier = custom_identifier
        self.cap = capacity
        # intialize base class
        process = 'main_logic'
        super().__init__(env=env, name=name, process=process, **kwargs)
        
        # add logic queues
        # each resource uses one associated logic queue, logic queues are not physically available
        queue_name: str = f"queue_{self.name()}"
        self.logic_queue: Queue = sim.Queue(name=queue_name, env=self.env)
        
        # currently available jobs on that resource
        self.contents: OrderedDict[ObjectID, Job] = OrderedDict()
        
        # resource state parameters
        self.is_waiting: bool = False
        self.is_failed: bool = False
                
        # time in state parameters
        self.time_waiting: float = 0.
        self.time_failed: float = 0.
        
        # maybe for future, curently no working time calendars planned
        self.is_paused: bool = False
        self.time_paused: float = 0.
        
        # number of inputs/outputs
        self.num_inputs: int = 0
        self.num_outputs: int = 0
        
    @property
    def env_id(self) -> EnvID:
        return self._env_id
    
    def add_content(
        self,
        job: Job,
    ) -> None:
        """add contents to the InfrastructureObject"""
        job_id = job.job_id
        if job_id not in self.contents:
            self.contents[job_id] = job
        else:
            raise KeyError(f"Job {job} already in contents of {self}")
    
    def remove_content(
        self,
        job: Job,
    ) -> None:
        """remove contents from the InfrastructureObject"""
        job_id = job.job_id
        if job_id not in self.contents:
            del self.contents[job_id]
        else:
            raise KeyError(f"Job {job} not in contents of {self}")
    
    def put_job(
        self,
        job: Job,
    ) -> InfrastructureObject:
        """
        placing
        """
        # ALLOCATION REQUEST
        ## call dispatcher --> request for allocation
        ## self._dispatcher.request_allocation ...
        ### input job
        #### LATER: LOGIC FOR RESOURCE ALLOCATION (AGENT)
        ### - Dispatcher calls "get_next_operation"
        ### - Dispatcher returns target_machine
        ## ret: obtaining target machine
        # ++++++++++ add later ++++++++++++
        ## time component: given start date of operation
        ## returning release date, waiting for release date or release early
        target_proc_station = self.env.dispatcher.request_job_allocation(job=job)
            
        logic_queue = target_proc_station.logic_queue
        # check if associated buffers exist
        logger_prodStations.debug(f"[{self}] Check for buffers")
        buffers = target_proc_station.buffers
        if buffers:
            logger_prodStations.debug(f"[{self}] Buffer found")
            yield self.to_store(store=buffers, item=job, fail_delay=FAIL_DELAY)
            if self.failed():
                raise UserWarning(f"Store placement failed after {FAIL_DELAY} time steps. \
                    There seems to be deadlock.")
            # activate buffer for state definition process
            for buffer in buffers:
                buffer.activate()
        else:
            # adding request to machine
            # currently not possible because machines are components,
            # but resources which could be requested are not
            pass
        
        # load object: enter logic queue after physical placement
        job.enter(logic_queue)
        
        # activate target provessing station if passive
        if target_proc_station.ispassive():
            target_proc_station.activate()
        
        logger_prodStations.debug(f"[{self}] Put Job {job} in queue {target_proc_station.logic_queue}")
        
        return target_proc_station
    
    ### PROCESS LOGIC
    # each method of 'pre_process', 'sim_control', 'post_process' must be implemented in the child classes
    def pre_process(self) -> None:
        """return type: tuple with parameters or None"""
        raise NotImplementedError(f"No pre-process method for {self} of type {self.__class__.__name__} defined.")
    
    def sim_control(self) -> None:
        """return type: tuple with parameters or None"""
        raise NotImplementedError(f"No sim-control method for {self} of type {self.__class__.__name__} defined.")
    
    def post_process(self) -> None:
        """return type: tuple with parameters or None"""
        raise NotImplementedError(f"No post-process method for {self} of type {self.__class__.__name__} defined.")
    
    def main_logic(self) -> Iterator[Any]:
        """main logic loop for all resources in the simulation environment"""
        logger.debug(f"----> Process logic of {self}")
        # pre control logic
        ret = self.pre_process()
        # main control logic
        if ret is not None:
            ret = yield from self.sim_control(*ret)
        else:
            ret = yield from self.sim_control()
        # post control logic
        if ret is not None:
            ret = self.post_process(*ret)
        else:
            ret = self.post_process()

<a id='processingstation'></a>

In [577]:
class ProcessingStation(InfrastructureObject):
    
    def __init__(
        self,
        buffers: Iterable[Buffer] | None = None,
        **kwargs,
    ) -> None:
        """
        env: simulation environment in which the infrastructure object is embedded
        capacity: capacity of the infrastructure object, if multiple processing \
            slots available at the same time > 1, default=1
        """
        # intialize base class
        super().__init__(**kwargs)
        
        # add physical buffers, more than one allowed
        # contrary to logic queues buffers are infrastructure objects and exist physically
        if buffers is None:
            self._buffers: set[Buffer] = set()
        else:
            self._buffers: set[Buffer] = set(buffers).copy()
        
        # add processing station to the associated ones of each buffer
        # necessary because if the number of resources for one buffer exceeds its capacity
        # deadlocks are possible
        for buffer in self._buffers:
            buffer.add_prod_station(prod_station=self)
        
        
        # resource state parameters
        self.is_occupied: bool = False
        self.is_blocked: bool = False
        
        # time in state parameters
        self.time_occupied: float = 0.
        self.time_blocked: float = 0.
        
    @property
    def buffers(self) -> set[Buffer]:
        return self._buffers
    
    def add_buffer(
        self,
        buffer: Buffer,
    ) -> None:
        """
        adding buffer to the current associated ones
        """
        # only buffer types allowed
        if not isinstance(buffer, Buffer):
            raise TypeError(f"Object is no Buffer type. Only objects of type Buffer can be added as buffers.")
        # check if already present
        if buffer not in self._buffers:
            self._buffers.add(buffer)
            buffer.add_prod_station(prod_station=self)
        else:
            logger_prodStations.warning(f"The Buffer >>{buffer}<< is already associated with the resource >>{self}<<. \
                Buffer was not added to the resource.")

    def remove_buffer(
        self,
        buffer: Buffer,
    ) -> None:
        """
        removing buffer from the current associated ones
        """
        if buffer in self._buffers:
            self._buffers.remove(buffer)
            buffer.remove_prod_station(prod_station=self)
        else:
            raise KeyError(f"The buffer >>{buffer}<< is not associated with the resource >>{self}<< and \
                therefore could not be removed.")
            
    def get_job(self) -> Job:
        """
        getting jobs from associated predecessor resources
        """
        # entering target machine (logic_buffer)
        ## logic_buffer: job queue regardless of physical buffers
        ### entity physically on machine, but no true holding resource object (violates load-resource model)
        ### no capacity restrictions between resources, e.g. source can endlessly produce entities
        ## --- logic ---
        ## job enters logic queue of machine with unrestricted capacity
        ## each machine can have an associated physical buffer
        
        # request job from associated queue
        job = self.env.dispatcher.request_job_sequencing(req_obj=self)
        
        # request and get job from associated buffer if it exists
        if self._buffers:
            yield self.from_store(store=self._buffers, filter=lambda item: item.job_id == job.job_id)
        else:
            pass
        
        return job
    
    ### PROCESS LOGIC
    def pre_process(self) -> None:
        return None
    
    def sim_control(self) -> None:
        while True:
            # initialise state by passivating machines
            # resources are activated by other resources
            if len(self.logic_queue) == 0:
                yield self.passivate()
            logger_prodStations.debug(f"[MACHINE: {self}] is getting job from queue")
            
            ########
            # GET FUNCTION PARENT CLASS
            # ONLY PROCESSING STATIONS ARE ASKING FOR SEQUENCING
            job = yield from self.get_job()
            
            # theoretically request to dispatcher possible
            # request job from associated queue
            #job = self.env.dispatcher.request_job_sequencing(req_obj=self)
            
            # get current operation instance
            current_op = job.current_op
            # get processing time of the current operation
            # time set by assigning the new next oepration when exiting an infrastructure object
            proc_time = job.current_proc_time
            
            logger_prodStations.debug(f"[START] job ID {job.job_id} at {self.env.now()} on machine ID {self.custom_identifier} \
                with proc time {proc_time}")
            
            # RELEVANT INFORMATION BEFORE PROCESSING
            current_op.start_time = self.env.now()
            # PROCESSING
            yield self.hold(proc_time)
            # RELEVANT INFORMATION AFTER PROCESSING
            # [ADDING] new function to update job properties
            current_op.end_time = self.env.now()
            current_op.is_finished = True
            job.num_finished_ops += 1
            self.env.dispatcher.write_op_times(op_id=current_op.op_id)
            
            
            logger_prodStations.debug(f"[END] job ID {job.job_id} at {self.env.now()} on machine ID {self.custom_identifier}")
            
            # only place job if there are open operations left
            # maybe add to 'put_job' method
            if job.open_operations:
                target_proc_station = yield from self.put_job(job=job)
            else:
                # ADD SINK LOGIC
                pass
            
            # return back control to job instance
            #job.activate()
    
    def post_process(self) -> None:
        return None

<a id='machine'></a>

In [578]:
class Machine(ProcessingStation):
    
    def __init__(
        self,
        resource_type: str = 'Machine',
        **kwargs,
    ) -> None:
        """
        env:        SimPy Environment in which machine is embedded
        num_slots:  capacity of the machine, if multiple processing 
                    slots available at the same time > 1, default=1
        """
        # assert object information
        self.res_type = resource_type
        
        # intialize base class
        super().__init__(**kwargs)
        
    ### STATE SETTING
    

<a id='buffer'></a>

In [579]:
from salabim.salabim import Component, Queue


class Buffer(sim.Store, InfrastructureObject):
    
    def __init__(
        self,
        capacity: float,
        resource_type: str = 'Buffer',
        **kwargs,
    ) -> None:
        """
        capacity: capacity of the buffer, can be infinite
        """
        # assert object information
        self.res_type = resource_type
        
        # intialize base classes
        # using hard-coded classes because salabim does not provide 
        # interfaces for multiple inheritance
        sim.Store.__init__(self, capacity=capacity, env=env)
        InfrastructureObject.__init__(self, capacity=capacity, **kwargs)
        
        # material flow relationships
        self._associated_prod_stations: set[ProcessingStation] = set()
        self._count_associated_prod_stations: int = 0
        
        # resource state parameters
        self.is_full: bool = False
        self.is_empty: bool = False
    
    ### MATERIAL FLOW RELATIONSHIP
    def add_prod_station(
        self,
        prod_station: ProcessingStation
    ) -> None:
        """
        function to add processing stations which are associated with 
        """
        if not isinstance(prod_station, ProcessingStation):
            raise TypeError(f"Object is no ProcessingStation type. Only objects of type ProcessingStation can be added to a buffer.")
        
        # check if adding a new resource exceeds the given capacity
        # each associated processing station needs one storage place in the buffer
        # else deadlocks are possible
        if (self._count_associated_prod_stations + 1) > self.cap:
            raise UserWarning(f"Tried to add a new resource to buffer {self}, but the number of associated \
                resources exceeds its capacity which could result in deadlocks.")
        
        # check if processing station can be added
        if prod_station not in self._associated_prod_stations:
            self._associated_prod_stations.add(prod_station)
            self._count_associated_prod_stations += 1
        else:
            logger_prodStations.warning(f"The Processing Station >>{prod_station}<< is already associated with the resource >>{self}<<. \
                Processing Station was not added to the resource.")
        
    def remove_prod_station(
        self,
        prod_station: ProcessingStation
    ) -> None:
        """
        removing a processing station from the current associated ones
        """
        if prod_station in self._associated_prod_stations:
            self._associated_prod_stations.remove(prod_station)
            self._count_associated_prod_stations -= 1
        else:
            raise KeyError(f"The processing station >>{prod_station}<< is not associated with the resource >>{self}<< and \
                therefore could not be removed.")
    
    ### PROCESS LOGIC
    def pre_process(self) -> None:
        return None
    
    def sim_control(self) -> None:
        while True:
            logger_prodStations.debug(f"[BUFFER: {self}] Invoking at {self.env.now()}")
            # full
            if self.available_quantity() == 0:
                self.is_full = True
                self.is_empty = False
                logger_prodStations.debug(f"[BUFFER: {self}] Set to 'FULL' at {self.env.now()}")
            # empty
            elif self.available_quantity() == self.capacity():
                self.is_full = False
                self.is_empty = True
                logger_prodStations.debug(f"[BUFFER: {self}] Set to 'EMPTY' at {self.env.now()}")
            else:
                self.is_full = False
                self.is_empty = False
                logger_prodStations.debug(f"[BUFFER: {self}] Neither 'EMPTY' nor 'FULL' at {self.env.now()}")
            
            yield self.passivate()
        
        return None
            
    def post_process(self) -> None:
        return None
    

[Jump to top](#top)

In [580]:
random.normalvariate(10, 2)

10.490652683415727

**Sources:**
- entity generation:
    - constant
    - random

<a id='source'></a>

In [596]:
class Source(InfrastructureObject):
    
    def __init__(
        self,
        resource_type: str = 'Source',
        proc_time: float = 1.,
        random_generation: bool = False,
        job_generator: RandomJobGenerator | None = None,
        num_gen_jobs: int = 5,
        **kwargs,
    ) -> None:
        """
        num_gen_jobs: total number of jobs to be generated
        """
        # assert object information and register object in the environment
        self.res_type = resource_type
        
        # random generation
        if random_generation and job_generator is None:
            raise ValueError("Random generator instance needed for random job generation")
        
        self.random_generation = random_generation
        self.job_generator = job_generator
        
        ### REWORK
        # initialize component with necessary process function
        random.seed(42)
        super().__init__(**kwargs)
        
        # parameters
        self.proc_time = proc_time
        self.num_gen_jobs = num_gen_jobs
        
    
    ### STATE SETTING
    
    
    def _obtain_proc_time(self) -> float:
        """
        function to generate a constant or random processing time
        """
        if self.random_generation:
            # random generation, add later
            return self.proc_time
        else:
            return self.proc_time
    
    ### PROCESS LOGIC
    def pre_process(self) -> None:
        return None
    
    def sim_control(self) -> None:
        # id counter for debugging, else endless generation
        id_count = 0
        while id_count < self.num_gen_jobs:
            # start at t=0 with generation
            
            # generate object
            ## random job properties
            ## currently: each job passes each machine, only one machine of each operation type
            mat_ProcTimes, mat_JobMachID = self.job_generator.gen_rnd_job(self.env.num_proc_stations)
            job = Job(dispatcher=self.env.dispatcher, proc_times=mat_ProcTimes.tolist(), 
                      machine_order=mat_JobMachID.tolist(), status='generated')
            logger_sources.debug(f"[SOURCE: {self}] Generated {job} at {self.env.now()}")
            
            
            
            
            logger_sources.debug(f"[SOURCE: {self}] Request allocation...")
            # put job via 'put_job' function, implemented in parent class 'InfrastructureObject'
            target_proc_station = yield from self.put_job(job=job)
            logger_sources.debug(f"[SOURCE: {self}] PUT JOB with ret = {target_proc_station}")
            
            # hold for defined generation time (constant or statistically distributed)
            # if hold time elapsed start new generation
            proc_time = self._obtain_proc_time()
            logger_sources.debug(f"[SOURCE: {self}] Hold for >>{proc_time}<< at {self.env.now()}")
            yield self.hold(proc_time)
            # set counter up
            id_count += 1
            
        return None
            
    def post_process(self) -> None:
        return None
    

In [606]:
env = SimulationEnvironment(name='base')
job_generator = RandomJobGenerator(seed=2)
dispatcher = Dispatcher(env=env, priority_rule='FIFO')
#buffer = Buffer(capacity=INF, env=env, custom_identifier=10)

# resources
for machine in range(5):
    buffer = Buffer(capacity=5, env=env, custom_identifier=(10+machine))
    MachInst = Machine(env=env, custom_identifier=machine, buffers=[buffer])
    #MachInst = Machine(env=env, custom_identifier=machine)

source = Source(env=env, custom_identifier='quelle', proc_time=1., 
                random_generation=True, job_generator=job_generator, num_gen_jobs=10)


In [607]:
dispatcher.curr_prio_rule = 'SPT'

INFO:dispatcher:Changed priority rule to SPT


In [608]:
env.name()

'base'

In [609]:
env.run()

DEBUG:base:----> Process logic of Buffer (Buffer_env_1)
DEBUG:prodStations:[BUFFER: Buffer (Buffer_env_1)] Invoking at 0.0
DEBUG:prodStations:[BUFFER: Buffer (Buffer_env_1)] Set to 'EMPTY' at 0.0
DEBUG:base:----> Process logic of Machine (Machine_env_2)
DEBUG:base:----> Process logic of Buffer (Buffer_env_3)
DEBUG:prodStations:[BUFFER: Buffer (Buffer_env_3)] Invoking at 0.0
DEBUG:prodStations:[BUFFER: Buffer (Buffer_env_3)] Set to 'EMPTY' at 0.0
DEBUG:base:----> Process logic of Machine (Machine_env_4)
DEBUG:base:----> Process logic of Buffer (Buffer_env_5)
DEBUG:prodStations:[BUFFER: Buffer (Buffer_env_5)] Invoking at 0.0
DEBUG:prodStations:[BUFFER: Buffer (Buffer_env_5)] Set to 'EMPTY' at 0.0
DEBUG:base:----> Process logic of Machine (Machine_env_6)
DEBUG:base:----> Process logic of Buffer (Buffer_env_7)
DEBUG:prodStations:[BUFFER: Buffer (Buffer_env_7)] Invoking at 0.0
DEBUG:prodStations:[BUFFER: Buffer (Buffer_env_7)] Set to 'EMPTY' at 0.0
DEBUG:base:----> Process logic of Machine 

DEBUG:prodStations:[START] job ID 0 at 0.0 on machine ID 0                 with proc time 4
INFO:dispatcher:Successfully registered job with JobID 1 and name J_gen_1
INFO:dispatcher:Successfully registered operation with OpID 5 and name O_gen_5
INFO:dispatcher:Successfully registered operation with OpID 6 and name O_gen_6
INFO:dispatcher:Successfully registered operation with OpID 7 and name O_gen_7
INFO:dispatcher:Successfully registered operation with OpID 8 and name O_gen_8
INFO:dispatcher:Successfully registered operation with OpID 9 and name O_gen_9
DEBUG:sources:[SOURCE: Source (Source_env_11)] Generated Job (J_gen_1) at 1.0
DEBUG:sources:[SOURCE: Source (Source_env_11)] Request allocation...
INFO:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] REQUEST TO DISPATCHER FOR ALLOCATION
DEBUG:dispatcher:[DISPATCHER: Dispatcher(env: base, EnvID: 0)] Next operation is Operation(ProcTime: 9, MachineID: 2) with machine group (machine) Machine (Machine_env_6)
DEBUG:prodStations:[So

**Next Step:**
- add output process to processing stations (or infrastructure objects)

In [585]:
dispatcher.op_db

,job_id,job_name,custom_id,op,name,machine,machine_custom_id,machine_name,duration,start_time,end_time,status
op_id,,,,,,,,,,,,
0,0,J_gen_0,None,"Operation(ProcTime: 4, MachineID: 1)",O_gen_0,Machine (Machine_env_4),1,Machine_env_4,4.0,0.0,4.0,None
1,0,J_gen_0,None,"Operation(ProcTime: 8, MachineID: 0)",O_gen_1,Machine (Machine_env_2),0,Machine_env_2,8.0,14.0,22.0,None
2,1,J_gen_1,None,"Operation(ProcTime: 4, MachineID: 0)",O_gen_2,Machine (Machine_env_2),0,Machine_env_2,4.0,1.0,5.0,None
3,1,J_gen_1,None,"Operation(ProcTime: 1, MachineID: 1)",O_gen_3,Machine (Machine_env_4),1,Machine_env_4,1.0,5.0,6.0,None
4,2,J_gen_2,None,"Operation(ProcTime: 7, MachineID: 0)",O_gen_4,Machine (Machine_env_2),0,Machine_env_2,7.0,22.0,29.0,None
5,2,J_gen_2,None,"Operation(ProcTime: 4, MachineID: 1)",O_gen_5,Machine (Machine_env_4),1,Machine_env_4,4.0,29.0,33.0,None
6,3,J_gen_3,None,"Operation(ProcTime: 4, MachineID: 0)",O_gen_6,Machine (Machine_env_2),0,Machine_env_2,4.0,29.0,33.0,None
7,3,J_gen_3,None,"Operation(ProcTime: 5, MachineID: 1)",O_gen_7,Machine (Machine_env_4),1,Machine_env_4,5.0,33.0,38.0,None
8,4,J_gen_4,None,"Operation(ProcTime: 9, MachineID: 0)",O_gen_8,Machine (Machine_env_2),0,Machine_env_2,9.0,5.0,14.0,None


In [605]:
fig = dispatcher.draw_gantt_chart(use_custom_proc_station_id=True, sort_by_proc_station=True)

In [610]:
fig = dispatcher.draw_gantt_chart(use_custom_proc_station_id=True, sort_by_proc_station=True)

*Priority rules: prototyping*

In [573]:
queue = MachInst.logic_queue.copy()

In [540]:
ret = queue.tail()
ret

Job (J_gen_11)

In [541]:
qu = queue.as_list()
qu

[Job (J_gen_0), Job (J_gen_1), Job (J_gen_9), Job (J_gen_11)]

In [503]:
q = queue.as_list()

In [509]:
ret = q.pop()
ret

Job (J_gen_5)

In [ ]:
# FIFO is simple pop from the queue
job = queue.pop()

In [ ]:
# LIFO is simple pop from the queue's end [-1]
job = queue.pop(-1)

In [ ]:
l = queue.as_list()
l

In [ ]:
from operator import itemgetter, attrgetter

In [ ]:
for item in l:
    print(f"{item.current_proc_time}")

In [ ]:
#%%timeit
# demo shortest processing time
# sort descending and pop last item
ret = sorted(queue, key=attrgetter('current_proc_time'), reverse=True)
job = ret.pop()
# remove job from original queue
queue.remove(job)

In [ ]:
env.dispatcher.job_db

In [ ]:
env.dispatcher.op_db

[Jump to top](#top)

---

#### **Adding database approach also to the job dispatcher**
**properties**:
- job_id
- custom_id
- job instance
- name
- product_type (for later implementation of different product types)

<a id='dispatcher'></a>

In [590]:
class Dispatcher(object):
    
    def __init__(
        self,
        env: SimulationEnvironment,
        priority_rule: str = 'FIFO',
    ) -> None:
        """
        Dispatcher class for given environment (only one dispatcher for each environment)
        - different functions to monitor all jobs in the environment
        - jobs report back their states to the dispatcher
        """
                
        # job data base as simple Pandas DataFrame
        # column data types
        self._job_prop: dict[str, type] = {
            'job_id': int,
            'custom_id': object,
            'job': object,
            'name': str,
            'job_type': str,
            'status': str,
        }
        self._job_db: DataFrame = pd.DataFrame(columns=list(self._job_prop.keys()))
        self._job_db: DataFrame = self._job_db.astype(self._job_prop)
        self._job_db: DataFrame = self._job_db.set_index('job_id')
        self._job_lookup_props: set[str] = set(['job_id', 'custom_id', 'name'])
        
        # operation data base as simple Pandas DataFrame
        # column data types
        self._op_prop: dict[str, type] = {
            'op_id': int,
            'job_id': int,
            'job_name': str,
            'custom_id': object,
            'op': object,
            'name': str,
            'machine': object,
            'machine_custom_id': object,
            'machine_name': str,
            'duration': float,
            'start_time': float,
            'end_time': float,
            'status': str,
        }
        self._op_db: DataFrame = pd.DataFrame(columns=list(self._op_prop.keys()))
        self._op_db: DataFrame = self._op_db.astype(self._op_prop)
        self._op_db: DataFrame = self._op_db.set_index('op_id')
        self._op_lookup_props: set[str] = set(['op_id', 'job_id', 'custom_id', 'name', 'machine'])
                
        # register in environment and get EnvID
        self._env = env
        self._env_id: EnvID = self._env.register_dispatcher(self)
        
        ########## PERHAPS REWORK
        self._disposable_jobs: dict[int, Job] = dict()
        self.job_pool: OrderedDict[ObjectID, Job] = OrderedDict()
        # managing IDs
        self._id_types = set(['job', 'op'])
        self._job_id_counter: ObjectID = 0
        self._op_id_counter: ObjectID = 0
        
        # priority rules
        self._priority_rules: set[str] = set([
            'FIFO',
            'LIFO',
            'SPT',
            'LPT',
        ])
        # set current priority rule
        if priority_rule not in self._priority_rules:
            raise ValueError(f"Priority rule {priority_rule} unknown. Must be one of {self._priority_rules}")
        else:
            self._curr_prio_rule = priority_rule
    
    ### DATA MANAGEMENT
    def __repr__(self) -> str:
        return f"Dispatcher(env: {self.env.name()}, EnvID: {self._env_id})"
    
    @property
    def env_id(self) -> EnvID:
        return self._env_id
    
    @property
    def env(self) -> SimulationEnvironment:
        return self._env
    
    @property
    def curr_prio_rule(self) -> str:
        return self._curr_prio_rule
    
    @curr_prio_rule.setter
    def curr_prio_rule(self, rule) -> None:
        if rule not in self._priority_rules:
            raise ValueError(f"Priority rule {rule} unknown. Must be one of {self._priority_rules}")
        else:
            self._curr_prio_rule = rule
            logger_dispatcher.info(f"Changed priority rule to {rule}")
    
    def _obtain_job_id(self) -> ObjectID:
        """Simple counter function for managing job IDs"""
        # assign id and set counter up
        job_id = self._id_counter
        self._id_counter += 1
        
        return job_id
    
    def _obtain_op_id(self) -> ObjectID:
        """Simple counter function for managing operation IDs"""
        # assign id and set counter up
        op_id = self._op_id_counter
        self._op_id_counter += 1
        
        return op_id
    
    def _obtain_load_obj_id(
        self,
        load_type: str,
    ) -> ObjectID:
        """Simple counter function for managing operation IDs"""
        # assign id and set counter up
        
        if load_type not in self._id_types:
            raise ValueError(f"Given type {type} not valid. Choose from '{self._id_types}'")
        
        match load_type:
            case 'job':
                ident_no = self._job_id_counter
                self._job_id_counter += 1
            case 'op':
                ident_no = self._op_id_counter
                self._op_id_counter += 1
        
        return ident_no
    
    def register_job(
        self,
        obj: Job,
        custom_identifier: CustomID | None,
        name: str | None,
        status: str | None,
    ) -> tuple[SimulationEnvironment, ObjectID, str]:
        """
        registers an job object in the dispatcher instance by assigning an unique id and 
        adding the object to the associated jobs
        
        object:     env resource
        returns:
            env_id: assigned env ID
        """
        # obtain id
        job_id = self._obtain_load_obj_id(load_type='job')
        
        # custom name
        if name is None:
            name = f'J_gen_{job_id}'
        
        # new entry for job data base
        new_entry: DataFrame = pd.DataFrame({
                                'job_id': [job_id],
                                'custom_id': [custom_identifier],
                                'job': [obj],
                                'name': [name],
                                'job_type': [obj.job_type],
                                'status': status})
        new_entry: DataFrame = new_entry.astype(self._job_prop)
        new_entry: DataFrame = new_entry.set_index('job_id')
        self._job_db: DataFrame = pd.concat([self._job_db, new_entry])
        
        logger_dispatcher.info(f"Successfully registered job with JobID {job_id} and name {name}")
        
        return self.env, job_id, name
    
    def register_operation(
        self,
        obj: Operation,
        machine_identifier: MachineID,
        custom_identifier: CustomID | None,
        name: str | None,
        status: str | None, ###### add status types later
    ) -> tuple[ObjectID, str, Machine]: ##### add machine type later
        """
        registers an operation object in the dispatcher instance by assigning an unique id and 
        adding the object to the associated operations
        
        obj: operation to register
        machine_identifier: custom ID of the associated machine (user interface)
        custom_identifier: custom identifier of the operation 
            (kept for consistency reasons, perhaps remove later)
        name: assigned name the operation
        status: for future features if status of operations is tracked
        
        outputs:
        op_id: assigned operation ID
        name: assigned name
        machine: corresponding machine infrastructure object
        """
        # obtain id
        op_id = self._obtain_load_obj_id(load_type='op')
        
        # custom name
        if name is None:
            name = f'O_gen_{op_id}'
        
        # corresponding machine object on which operation is performed
        machine = self._env.get_res_obj_by_prop(property='custom_id', val=machine_identifier)
        
        # new entry for operation data base
        new_entry: DataFrame = pd.DataFrame({
                                'op_id': [op_id],
                                'job_id': [obj.job_id],
                                'job_name': [obj.job.name()],
                                'custom_id': [custom_identifier],
                                'op': [obj],
                                'name': [name],
                                'machine': [machine],
                                'machine_custom_id': [machine.custom_identifier],
                                'machine_name': [machine.name()],
                                'duration': [obj.proc_time],
                                'start_time': [obj.start_time],
                                'end_time': [obj.end_time],
                                'status': status})
        new_entry: DataFrame = new_entry.astype(self._op_prop)
        new_entry: DataFrame = new_entry.set_index('op_id')
        self._op_db: DataFrame = pd.concat([self._op_db, new_entry])
        
        logger_dispatcher.info(f"Successfully registered operation with OpID {op_id} and name {name}")
        
        # return machine object
        return op_id, name, machine
    
    def write_op_times(
        self,
        op_id: ObjectID,
    ) -> None:
        """
        add start and end time information to the operation database 
        """
        # get operation instance
        curr_op: Operation = self._op_db.loc[op_id, 'op']
        # write operation properties to operation database
        self._op_db.loc[op_id, 'start_time'] = curr_op.start_time
        self._op_db.loc[op_id, 'end_time'] = curr_op.end_time
    
    # vvvv LEGACY vvvv
    # special function for custom random generator of JSSP instances
    def gen_job_pool_generic(
        self,
        mat_ProcTimes: npt.NDArray[np.uint16],
        mat_JobMachID: npt.NDArray[np.uint16],
    ) -> OrderedDict[ObjectID, Job]:
        """
        function to build a integrated job pool if generic JxM JSSP instances are used
        mat_ProcTimes: matrix of processing times | shape=(n_jobs,n_machines)
        mat_JobMachID: matrix of machine IDs per job starting by index 1 | shape=(n_jobs,n_machines)
        mat_OpID: matrix of operation IDs starting by index 1 | shape=(n_jobs,n_machines)
        """
            
        for job_id in range(len(mat_ProcTimes)):
            temp1 = mat_ProcTimes[job_id].tolist()
            temp2 = mat_JobMachID[job_id].tolist()
            JobInst = Job(
                dispatcher=self,
                proc_times=temp1,
                machine_order=temp2,
                custom_identifier=None,
                name=None,
            )
            ######### NOT NECESSARY ANYMORE BECAUSE JOBS REGISTER THEMSELVES
            # IN THE DISPATCHER INSTANCE AND THERFORE IN THE JOB DATABASE
            ######### ADD TO JOB (bottom-up approach) #######################
            ### jobs add themselves to job pool
            self.job_pool[job_id] = JobInst
            
        return self.job_pool
     
    @property
    def job_db(self) -> DataFrame:
        """
        obtain a current overview of registered jobs in the environment
        """
        return self._job_db
    
    @property
    def op_db(self) -> DataFrame:
        """
        obtain a current overview of registered operations in the environment
        """
        return self._op_db

    #@lru_cache(maxsize=200)
    def get_job_obj_by_prop(
        self,
        property: str, 
        val: EnvID | CustomID | str,
        target_prop: str = 'job',
    ) -> Job:
        """
        obtain a job object from the dispatcher by its property and corresponding value
        properties: job_id, custom_id, name
        """
        # check if property is a filter criterion
        if property not in self._job_lookup_props:
            raise IndexError(f"Property '{property}' is not allowed. Choose from {self._job_lookup_props}")
        # None type value can not be looked for
        if val is None:
            raise TypeError("The lookup value can not be of type 'None'.")
        
        # filter resource database for prop-value pair
        if property == 'job_id':
            # direct indexing for ID property; job_id always unique, no need for duplicate check
            ###############
            try:
                temp1: Job = self._job_db.loc[val, target_prop]
                return temp1
            except KeyError:
                raise IndexError(f"There were no resources found for the property '{property}' \
                                with the value '{val}'")
        else:
            temp1: Series = self._job_db.loc[self._job_db[property] == val, target_prop]
            # check for empty search result, at least one result necessary
            if len(temp1) == 0:
                raise IndexError(f"There were no resources found for the property '{property}' \
                                with the value '{val}'")
            # check for multiple entries with same prop-value pair
            ########### PERHAPS CHANGE NECESSARY
            ### multiple entries but only one returned --> prone to errors
            elif len(temp1) > 1:
                # warn user
                logger_dispatcher.warning(f"CAUTION: There are multiple resources which share the \
                            same value '{val}' for the property '{property}'. \
                            Only the first entry is returned.")
            
            return temp1.iat[0]
    
    ### ROUTING LOGIC
    def request_job_allocation(
        self,
        job: Job,
    ) -> InfrastructureObject:
        """
        request an allocation decision for the given job 
        (determine the next processing station on which the job shall be placed)
        requester: output side infrastructure object
        request for: infrastructure object instance
        """
        # SIGNALING ALLOCATION DECISION
        # (ONLY IF PARALLEL PROCESSING STATIONS EXIST)
        ## theoretically: obtaining next operation --> information about machine group -->
        ## based on machine group: choice of corresponding allocation agent -->
        ## preparing feature vectors as input --> trigger agent decision -->
        ## map decision to processing station
        
        logger_dispatcher.info(f"[DISPATCHER: {self}] REQUEST TO DISPATCHER FOR ALLOCATION")
        
        # get the next operation of the job
        next_op = job.get_next_operation()
        target_proc_station = next_op.target_machine
        
        logger_dispatcher.debug(f"[DISPATCHER: {self}] Next operation is {next_op} with machine group (machine) {target_proc_station}")
        
        return target_proc_station
           
    def request_job_sequencing(
        self,
        req_obj: ProcessingStation
    ) -> Job:
        """
        request a sequencing decision for a given queue of the requesting resource
        requester: input side processing stations
        request for: job instance
        
        req_obj: requesting object (ProcessingStation)
        """
        # SIGNALING SEQUENCING DECISION
        # (ONLY IF MULTIPLE JOBS IN THE QUEUE EXIST)
        ## theoretically: get logic queue of requesting object --> information about feasible jobs -->
        ## [*] choice of sequencing agent (based on which properties?) --> preparing feature vector as input -->
        ## trigger agent decision --> map decision to feasible jobs
        ## [*] use implemented priority rules as intermediate step
        
        logger_dispatcher.debug(f"[DISPATCHER: {self}] REQUEST TO DISPATCHER FOR SEQUENCING")
        
        # get logic queue of requesting object
        # contains all feasible jobs for this resource
        logic_queue = req_obj.logic_queue
        # get job from logic queue with currently defined priority rule
        job = self.seq_priority_rule(queue=logic_queue)
        
        return job
    
    def seq_priority_rule(
        self,
        queue: Queue,
    ) -> Job:
        """apply priority rules to a pool of jobs"""
        match self._curr_prio_rule:
            case 'FIFO':
                job = queue.pop()
            case 'LIFO':
                job = queue.pop(-1)
            case 'SPT':
                # sort descending and pop last item
                temp = queue.as_list()
                temp = sorted(temp, key=attrgetter('current_proc_time'), reverse=True)
                job = temp.pop()
                # remove job from original queue
                queue.remove(job)
            case 'LPT':
                # sort asscending and pop last item
                temp = queue.as_list()
                temp = sorted(temp, key=attrgetter('current_proc_time'), reverse=False)
                job = temp.pop()
                # remove job from original queue
                queue.remove(job)
                
        return job
    
    ### ANALYSE
    def draw_gantt_chart(
        self,
        use_custom_proc_station_id: bool = True,
        sort_by_proc_station: bool = False,
        sort_ascending: bool = True,
        save_img: bool = False,
        save_html: bool = False,
        file_name: str = 'gantt_chart',
    ) -> PlotlyFigure:
        """
        draw a Gantt chart based on the dispatcher's operation database
        use_custom_machine_id: whether to use the custom IDs of the processing station (True) or its name (False)
        sort_by_proc_station: whether to sort by processing station property (True) or by job name (False) \
            default: False
        sort_ascending: whether to sort in ascending (True) or descending order (False) \
            default: True
        use_duration: plot each operation with its scheduled duration instead of the delta time \
            between start and end; if there were no interruptions both methods return the same results \
            default: False
        """
        # filter operation DB for relevant information
        df = self._op_db.filter(items=['job_name', 'machine_custom_id', 'machine_name', 'start_time', 'end_time', 'duration'])
        # calculate delta time between start and end
        df['delta'] = df['end_time'] - df['start_time']
        
        # sorting
        sort_key: str = ''
        # chose relevant processing station property
        proc_station_prop: str = ''
        if use_custom_proc_station_id:
            proc_station_prop = 'machine_custom_id'
        else:
            proc_station_prop = 'machine_name'
        
        
        # check if sorting by processing station is wanted and custom ID should be used or not
        if sort_by_proc_station:
            sort_key = proc_station_prop
        else:
            sort_key = 'job_name' 
        
        df = df.sort_values(by=sort_key, ascending=sort_ascending, kind='stable')
        
        # build Gantt chart with Plotly Timeline
        fig = px.timeline(df, x_start='start_time', x_end='end_time', 
                          y=proc_station_prop, color='job_name')
        fig.update_yaxes(type='category', autorange='reversed')
        fig.update_xaxes(type='linear')

        # reset axis scale for every figure element
        # https://stackoverflow.com/questions/66078893/plotly-express-timeline-for-gantt-chart-with-integer-xaxis
        for d in fig.data:
            filt = df['job_name'] == d.name
            d.x = df.loc[filt, 'delta']

        fig.show()
        
        if save_html:
            file = f'{file_name}.html'
            fig.write_html(file)
        
        if save_img:
            file = f'{file_name}.svg'
            fig.write_image(file)
        
        return fig
    
    ### DISPOSABLE JOBS
    ### STILL NECESSARY???
    def add_disposable_job(
        self,
        job: Job,
    ) -> None:
        """
        add job to the disposable ones
        """
        self._disposable_jobs[job.job_id] = job
    
    @property
    def disposable_jobs(self) -> dict[int, Job]:
        return self._disposable_jobs
    
    ################# REWORK ##################
    ### maybe add a corresponding property in the job DB
    def get_disposable_jobs(
        self,
        job_set: OrderedDict,
    ) -> tuple[list[ObjectID], list[Job]]:
        """
        function needs to be reworked, jobs should report back information to a dispatcher instance
        (bottom-up instead of top-down)
        """
        #########################################
        self._disposable_jobs_ID: list[int] = list()
        self._disposable_jobs: list[Job] = list()
        
        for job_id, job in job_set.items():
            if job.is_disposable:
                self._disposable_jobs_ID.append(job_id)
                self._disposable_jobs.append(job)
                
        return self._disposable_jobs_ID, self._disposable_jobs

[Jump to top](#top)

<a id='operation'></a>

In [523]:
class Operation(object):
    
    def __init__(
        self,
        dispatcher: Dispatcher,
        job: Job,
        proc_time: float,
        machine_identifier: MachineID,
        custom_identifier: CustomID | None = None,
        name: str | None = None,
        status: str | None = None,
    ) -> None:
        """
        identifier: operation's ID
        proc_times: operation's processing times
        machine_identifier: ID of machine on which operation is processed
        """
        # !!!!!!!!! perhaps processing times in future multiple entries depending on associated machine
        # change of input format necessary, currently only one machine for each operation
        # no groups, no differing processing times for different machines 

        # assert operation information
        self._dispatcher = dispatcher
        self._job = job
        self._job_id = job.job_id
        self._machine_identifier = machine_identifier
        
        # process information
        # processing time
        self.proc_time: float = proc_time
        # starting and end points
        # in future setting starting points in advance possible
        self.start_time: float = 0.
        self.end_time: float = 0.
        self.is_finished: bool = False
        
        ########### adding machine instances
        ### perhaps adding machine sets if multiple machines possible (machine groups)
        # assignment of machine instance by dispatcher
        # from dispatcher: op_id, name, target_machine
        self._op_id, self.name, self.target_machine = self.dispatcher.register_operation(
                                                        obj=self, machine_identifier=self._machine_identifier,
                                                        custom_identifier=custom_identifier, name=name, 
                                                        status=status)  
    
    def __repr__(self) -> str:
        return f"Operation(ProcTime: {self.proc_time}, MachineID: {self._machine_identifier})"    
    
    @property   
    def dispatcher(self) -> Dispatcher:
        return self._dispatcher
    
    @property
    def op_id(self) -> ObjectID:
        return self._op_id
    
    @property
    def job(self) -> Job:
        return self._job
    
    @property
    def job_id(self) -> ObjectID:
        return self._job_id

[Jump to top](#top)

<a id='job'></a>

In [524]:
class Job(sim.Component):
    
    def __init__(
        self,
        dispatcher: Dispatcher,
        proc_times: list[float],
        machine_order: list[MachineID],
        custom_identifier: CustomID | None = None,
        name: str | None = None,
        status: str | None = None,
        **kwargs,
    ) -> None:
        """
        ############## ADD DESCRIPTION
        """
        ### BASIC INFORMATION ###
        # assert job information
        self.custom_identifier = custom_identifier
        self.job_type: str = 'Job'
        self._dispatcher = dispatcher
        
        ### register job instance
        env, self._job_id, name = self._dispatcher.register_job(
                                    obj=self, custom_identifier=self.custom_identifier,
                                    name=name, status=status)
        
        # intialize base class
        #process = 'basic_logic'
        process = 'debug'
        super().__init__(env=env, name=name, process=process, **kwargs)
        
        ### OPERATIONS ##
        self.operations: deque[Operation] = deque()
        
        for idx, op_proc_time in enumerate(proc_times):
            Op = Operation(
                dispatcher=self._dispatcher,
                job=self,
                proc_time=op_proc_time,
                machine_identifier=machine_order[idx],
            )
            self.operations.append(Op)
            
        self.open_operations = self.operations.copy()
        self.total_num_ops: int = len(self.operations)
        self.num_finished_ops: int = 0
        # current OP and properties set by function "get_next_operation"
        self._current_op: Operation | None = None
        self._current_proc_time: float | None = None
        # rank-like property, set if job enters the infrastructure object
        # acts like a counter to allow easy sorting even if queue order is not maintained
        self._obj_entry_idx: int | None = None
        
        ### STATE ###
        # intra-process job state parameters
        # job is being processed, maybe better naming in future
        self.is_occupied: bool = False
        # waiting state only when released
        self.is_waiting: bool = False
        # if lying on failed machine
        self.is_failed: bool = False
        
        # intra-process time characteristics
        self.time_occupied: float = 0.
        self.time_waiting: float = 0.
        self.time_failed: float = 0.
        
        # inter-process job state parameters
        # first operation scheduled --> released job
        self.is_released: bool = False
        # job's next operation is disposable
        # true for each new job, maybe reworked in future for jobs with
        # a start date later than creation date
        self.is_disposable: bool = True
        # add job to disposable ones
        ret = self.dispatcher.add_disposable_job(self)
        # last operation ended --> finished job
        self.is_finished: bool = False
        
        # inter-process time characteristics
        # time of first operation starting point
        self.time_entry: float = 0.
        # time of last operation ending point
        self.time_exit: float = 0.
        
        # current resource location
        self.current_resource: InfrastructureObject | None = None
    
    @property
    def dispatcher(self) -> Dispatcher:
        return self._dispatcher
    
    @property
    def job_id(self) -> ObjectID:
        return self._job_id
    
    @property
    def current_op(self) -> Operation | None:
        """
        returns the current operation of the job
        If a job is currently being processed its current operation is 
        not changed until this operation is finished.
        """
        return self._current_op
    
    @property
    def current_proc_time(self) -> float | None:
        """
        returns the processing time of the current operation
        If a job is currently being processed its current processing time is 
        not changed until this operation is finished.
        """
        return self._current_proc_time
    
    @property
    def obj_entry_idx(self) -> int | None:
        """
        returns the entry index which is set by each infrastructure object
        """
        return self._obj_entry_idx
    
    def get_next_operation(self) -> Operation:
        """
        get next operation
        """
        op = self.open_operations.popleft()
        self._current_op = op
        self._current_proc_time = op.proc_time
        
        return op
    
    def has_job_id(
        self,
        job_id: ObjectID,
    ) -> bool:
        """
        checks whether the current job has the given id
        """
        if self._job_id == job_id:
            return True
        else:
            return False
    
    ### PROCESS LOGIC ###
    def basic_logic(self) -> Iterator[Any]:
        # perform routing as long as there are open operations
        # perhaps problem with sinks because there are no final operations for sinks
        #print('################### TEST #################')
        #print(f'Length open operations: {len(self.open_operations)}')
        while len(self.open_operations) != 0:
            yield self.hold(5)
            print(f'{self} greetings at {self.env.now()}')
            yield self.passivate()
            
            
    def debug(self) -> None:
        pass       
            
    
    def process(self) -> Iterator[Any]:
        # perform routing as long as there are open operations
        # perhaps problem with sinks because there are no final operations for sinks
        while len(self.open_operations) != 0:
                # get current operation
                curr_op = self.get_next_operation()
                
                ### REQUEST TO DISPATCHER (JOB SIDE: Allocation)
                # --> machine instance
                # target machine is more like a machine group with parallel machines
                machine = curr_op.target_machine
                
                print(f"Try putting Job-ID {self.job_id} \t Operation with ID {curr_op.op_id} \
                    on machine {machine}")
                
                #machine = self.dispatcher.get_machine_instance_by_id(machine_id)
                #print(f"Machine instance is {machine}")
                #print(f"Machine instance is: {machine} and ispassive {machine.ispassive()}")
                
                
                # entering associated resource queue
                self.enter(machine.buffer)
                if machine.ispassive():
                    machine.activate()
                yield self.passivate()
                # wait for processing
                #print(f"[END] Time now Job-ID {self.job_id} is {self.env.now()}")
                
                # after processing: write operation data
                # resource name, start, end, job name
                self._dispatcher.write_op_times(op_id=curr_op.op_id)

[Jump to top](#top)

---
#### Test simulation logic
- first main goal: execution of a generic example in a simple job shop with single machines and FIFO order

In [17]:
class TestDispatcher():
    def __init__(self) -> None:
        pass
    
    def add_dicts(self, job_dict, machine_dict):
        self.job_dict = job_dict
        self.machine_dict = machine_dict
        
    def get_machine_instance_by_id(self, ident):
        return self.machine_dict[ident]

In [18]:
f"buffer_{MachInst.name()}"

NameError: name 'MachInst' is not defined

In [ ]:
class TestMachine(sim.Component):
    def __init__(self, machine_id, environment, *args, **kwargs):
        super().__init__(env=environment, *args, **kwargs)
        self.machine_id = machine_id
        self.buffer = sim.Queue(env=environment)
        
    def process(self):
        while True:
            #yield self.passivate
            if len(self.buffer) == 0:
                yield self.passivate
            print("Machine is getting job from queue")
            # theoretically request to dispatcher possible
            job = self.buffer.pop()
            pro_time = job.pro_time
            print(f"Machine {self} got Job with ID {job.job_id} and ProcTime {pro_time}")
            #print(f"Time is {self.env.now()}")
            # processing
            self.hold(job.pro_time)
            job.activate()
    
        
class TestJob(sim.Component):
    def __init__(self, job_id, ops_proc_times, ops_machine_list, dispatcher, environment, machine_list, *args, **kwargs):
        super().__init__(env=environment, *args, **kwargs)
        self.job_id = job_id
        self.ops_proc_times = ops_proc_times.copy()
        self.ops_machine_list = ops_machine_list.copy()
        self.dispatcher = dispatcher
        self.ops_counter = 0
        self.pro_time = 5
        self.machine_list = machine_list.copy()
        
    def add_dicts(self, machine_dict):
        #self.job_dict = job_dict
        self.machine_dict = machine_dict
        
    def process(self):
        print(f"Job-ID {self.job_id} starts {env.now()}")
        while self.ops_counter < (len(self.ops_proc_times) - 1):
            print("Get Next operation...")
            #self.pro_time = self.ops_proc_times[self.ops_counter]
            machine_id = self.ops_machine_list[self.ops_counter]
            print(f"Operation with ProTime {self.pro_time} on machine {machine_id}")
            print("Obtain machine instance...")
            machine = self.dispatcher.get_machine_instance_by_id(machine_id)
            #machine = self.machine_dict[machine_id]
            #machine = self.machine_list.pop(0)
            print(f"Machine instance is: {machine} and ispassive {machine.ispassive()}")
            
            # now enter buffer
            self.enter(machine.buffer)
            # activate machine if passive
            #if machine.ispassive():
            machine.activate()
            yield self.passivate()
            
            
            self.ops_counter += 1
            print(f"Job-ID {self.job_id} ends {self.env.now()}")

In [ ]:
test_dispatcher = TestDispatcher()

In [ ]:
test_ProcTimes = mat_ProcTimes.copy()
#test_ProcTimes = np.expand_dims(test_ProcTimes, 0)

test_JobMachID = mat_JobMachID.copy()
#test_JobMachID = np.expand_dims(test_JobMachID, 0)

In [ ]:
class TestMachine(sim.Component):
    def __init__(self, machine_id, env, *args, **kwargs):
        super().__init__(env=env, *args, **kwargs)
        self.machine_id = machine_id
        self.buffer = sim.Queue(env=env)
    
    def process(self):
        while True:
            if len(self.buffer) == 0:
                yield self.passivate()
            job = self.buffer.pop()
            proc_time = job.proc_time
            print(f"[START] job ID {job.job_id} at {self.env.now()} on machine ID {self.machine_id} with proc time {job.proc_time}")
            yield self.hold(job.proc_time)
            print(f"[END] job ID {job.job_id} at {self.env.now()} on machine ID {self.machine_id}")
            job.activate()

class TestJob(sim.Component):
    def __init__(self, ident, machine_order, operation_procs, env, dispatcher, *args, **kwargs):
        super().__init__(env=env, *args, **kwargs)
        #self.machine_id = machine_id
        self.machine_list = machine_list.copy()
        self.proc_time = None
        self.counter = 0
        self.operation_procs = operation_procs.copy()
        #self.machine_dict = machine_dict.copy()
        #self.dict_keys = list(machine_dict.keys())
        self.machine_order = machine_order.copy()
        self.job_id = ident
        self.dispatcher = dispatcher
        
    def process(self):
        while len(self.operation_procs) != 0:
            #print(f"Job-ID {self.job_id} starts {env.now()}")
            print(f"Job-ID {self.job_id} \t Operation with ID {self.counter}")
            self.proc_time = self.operation_procs.pop(0)
            #machine = machine_list.pop(0)
            #machine = self.machine_dict[self.dict_keys[self.counter]]
            machine_id = self.machine_order.pop(0)
            machine = self.dispatcher.get_machine_instance_by_id(machine_id)
            #print(f"Machine instance is {machine}")
            #print(f"Machine instance is: {machine} and ispassive {machine.ispassive()}")
            self.enter(machine.buffer)
            if machine.ispassive():
                machine.activate()
            yield self.passivate()
            #print(f"[END] Time now Job-ID {self.job_id} is {self.env.now()}")
            self.counter += 1

In [ ]:
env = sim.Environment(trace=False)
machine_db = dict()
job_db = dict()
machine_list_inst = list()

for machine_id in np.unique(mat_JobMachID):
    machine = TestMachine(machine_id=machine_id.item(), env=env)
    machine_db[machine_id.item()] = machine
    machine_list_inst.append(machine)

for job_id, (proc_list, machine_list) in enumerate(zip(test_ProcTimes, test_JobMachID)):
    job = TestJob(ident=job_id, machine_order=machine_list.tolist(), operation_procs=proc_list.tolist(), 
                  dispatcher=test_dispatcher, env=env)
    #job.add_dicts(machine_db)
    job_db[job_id] = job
"""
proc_list = [10,8,6]
machine_list = [0,1,2]
job_id = 0
job = TestJob(ident=job_id, machine_dict=machine_db, operation_procs=proc_list,
                  dispatcher=test_dispatcher, env=env)
"""
   
test_dispatcher.add_dicts(job_db, machine_db)


In [ ]:
ret = job_db[0]

<a id='logic_test'></a>
#### Test Logic with integrated classes

In [ ]:
job_generator = RandomJobGenerator()
mat_ProcTimes, mat_JobMachID = job_generator.gen_rnd_JSSP_inst(2,3)

In [ ]:
env = SimulationEnvironment(name='base')

for machine in np.unique(mat_JobMachID):
    MachInst = Machine(env=env, custom_identifier=machine.item(), debug=False)

dispatcher = Dispatcher(env=env)

In [ ]:
test_ProcTimes = mat_ProcTimes.copy()
#test_ProcTimes = mat_ProcTimes[0,:]
#test_ProcTimes = np.expand_dims(test_ProcTimes, 0).copy()

test_JobMachID = mat_JobMachID.copy()
#test_JobMachID = mat_JobMachID[0,:]
#test_JobMachID = np.expand_dims(test_JobMachID, 0).copy()

***Deprecate job generation in dispatcher: dispatcher is only for managing existing jobs in the system***

In [ ]:
ret = dispatcher.gen_job_pool_generic(
    mat_ProcTimes=test_ProcTimes,
    mat_JobMachID=test_JobMachID,
)

In [ ]:
env.run()

In [ ]:
dispatcher.job_db

In [ ]:
job = dispatcher.get_job_obj_by_prop('job_id', val=0)

In [ ]:
job.job_id

In [ ]:
ident = 0
filter = lambda item: item.job_id == ident

In [ ]:
filter(job)

In [ ]:
type(filter)

In [ ]:
dispatcher.op_db

In [ ]:
fig = dispatcher.draw_gantt_chart(sort_by_machine_name=True, save_img=False, save_html=False)

##### Visualisation

In [ ]:
import plotly.express as px
import datetime

In [ ]:
dft_year = 1970
dft_month = 1
dft_day = 1

In [ ]:
start_time = 0
end_time = 5
date_obj_start = datetime.datetime(dft_year, dft_month, dft_day, hour=start_time)
date_obj_end = datetime.datetime(dft_year, dft_month, dft_day, hour=end_time)

In [ ]:
df = dispatcher.op_db

In [ ]:
df

In [ ]:
test = df.filter(items=['job_name', 'machine_name', 'start_time', 'end_time', 'duration'])

In [ ]:
test

In [ ]:
test['delta'] = test['end_time'] - test['start_time']

In [ ]:
test = test.sort_values('machine_name')

In [ ]:
type(fig)

In [ ]:
fig = px.timeline(test, x_start='start_time', x_end='end_time', y='machine_name', color='job_name')
fig.update_yaxes(autorange='reversed')
fig.layout.xaxis.type = 'linear'

for d in fig.data:
    filt = test['job_name'] == d.name
    #d.x = test[filt]['delta']
    d.x = test.loc[filt, 'duration']

fig.show()

In [ ]:
fig.data

In [ ]:
# https://stackoverflow.com/questions/66078893/plotly-express-timeline-for-gantt-chart-with-integer-xaxis
df = pd.DataFrame([
    dict(Resource="M1", Start=0, Finish=10, Job="J1"),
    dict(Resource="M2", Start=0, Finish=5, Job="J2"),
    dict(Resource="M1", Start=11, Finish=15, Job="J2")
])
df['delta'] = df['Finish'] - df['Start']

fig = px.timeline(df, x_start='Start', x_end='Finish', y='Resource', color='Job')
fig.update_yaxes(autorange='reversed')
fig.layout.xaxis.type = 'linear'

for d in fig.data:
    filt = df['Job'] == d.name
    d.x = df[filt]['delta']

fig.show()


In [ ]:
df

In [ ]:
for d in fig.data:
    print(d.name)
    filt = df['Job'] == d.name
    d.x = df[filt]['delta']
    print(d.x)

In [ ]:
fig.data[0].x = df[filt]['delta']

In [ ]:
fig.data[0]

In [ ]:
import plotly.express as px
import pandas as pd

df = pd.DataFrame([
    dict(Resource="Job A", Start=1, Finish=4),
    dict(Resource="Job B", Start=2, Finish=6),
    dict(Resource="Job A", Start=4, Finish=10)
])
df['delta'] = df['Finish'] - df['Start']

fig = px.timeline(df, x_start="Start", x_end="Finish", y="Resource")
fig.update_yaxes(autorange="reversed") 

fig.layout.xaxis.type = 'linear'
fig.data[0].x = df.delta.tolist()
#f = fig.full_figure_for_development(warn=False)
fig.show()

In [ ]:
df

In [ ]:
import plotly.figure_factory as ff

df = pd.DataFrame([
    dict(Task="Job A", Start=0, Finish=10, Resource="Alex"),
    dict(Task="Job B", Start=12, Finish=24, Resource="Alex"),
    dict(Task="Job C", Start=5, Finish=20, Resource="Max")
])

fig = ff.create_gantt(df, index_col = 'Resource',  bar_width = 0.4, show_colorbar=True)
fig.update_layout(xaxis_type='linear', autosize=True, width=800, height=400)
fig.show()

In [ ]:
df

**Link Collection**

- [Environment](#environment)
- [Machine](#machine)
- [Dispatcher](#dispatcher)
- [Operation](#operation)
- [Job](#job)
- [Logic Test](#logic_test)


[Jump to top](#top)